Calculate Area of Dog and use it to identify position in image

In [ ]:
import torch
from torchvision import models, transforms
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from PIL import Image
import numpy as np
import os
import csv
from tqdm import tqdm
import concurrent.futures
import shutil

weights = MaskRCNN_ResNet50_FPN_Weights.COCO_V1
model = models.detection.maskrcnn_resnet50_fpn(weights=weights)
model.eval()

def segment_dog(image_path):
    # Load and resize image
    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([transforms.ToTensor()])
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)

    # Detect objects and segment
    with torch.no_grad():
        predictions = model(img_tensor)

    # Ensure predictions contain 'masks'
    if 'masks' in predictions[0]:
        # Get the mask and bounding box of the dog with highest score
        masks = predictions[0]['masks']
        labels = predictions[0]['labels']
        boxes = predictions[0]['boxes']
        scores = predictions[0]['scores']

        # Label for dog in COCO dataset is 18
        dog_indices = [i for i, label in enumerate(labels) if label == 18]
        dog_scores = [scores[i] for i in dog_indices]

        if len(dog_scores) > 0:
            best_index = dog_indices[np.argmax(dog_scores)]
            best_box = boxes[best_index].numpy().astype(int)
            return best_box, img
        else:
            return None, img
    else:
        return None, img

def calculate_box_area(box):
    x1, y1, x2, y2 = box
    box_area = (x2 - x1) * (y2 - y1)
    return box_area

def calculate_image_area(image):
    width, height = image.size
    image_area = width * height
    return image_area

image_dir = '/content/drive/MyDrive/split_images/subfolder_1/'

csv_file_path = '/content/drive/MyDrive/Position/dog_area_percentage_1.csv'

processed_images = set()
if os.path.exists(csv_file_path):
    try:
        with open(csv_file_path, mode='r', newline='') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header
            for row in reader:
                processed_images.add(row[0])
    except StopIteration:
        pass

# List all image files
image_files = [filename for filename in os.listdir(image_dir) if filename.endswith(('.jpg', '.jpeg', '.png'))]

# Filter out already processed images
image_files = [f for f in image_files if os.path.splitext(f)[0] not in processed_images]

# Define batch size
batch_size = 8

# Function to process a batch of images
def process_batch(image_batch):
    batch_results = []
    for filename in image_batch:
        image_path = os.path.join(image_dir, filename)

        # Segment the dog and get the bounding box
        box, img = segment_dog(image_path)

        if box is not None:
            # Calculate the areas
            box_area = calculate_box_area(box)
            image_area = calculate_image_area(img)

            # Compute the percentage of the image occupied by the dog
            percentage = (box_area / image_area) * 100

            # Store the results
            dog_id = os.path.splitext(filename)[0]
            batch_results.append([dog_id, f"{percentage:.2f}"])
        else:
            # If no dog detected, store a row with 'No dog detected'
            dog_id = os.path.splitext(filename)[0]
            batch_results.append([dog_id, '0.00'])

    return batch_results

# Split image files into batches
image_batches = [image_files[i:i + batch_size] for i in range(0, len(image_files), batch_size)]

with open(csv_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)

    # Write header if the file is empty
    if os.path.getsize(csv_file_path) == 0:
        writer.writerow(['Dog ID', 'Percentage'])

    # Process batches asynchronously using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_batch, batch): batch for batch in image_batches}

        # Iterate over completed futures
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(image_batches), desc="Processing Batches"):
            try:
                batch_results = future.result()
                writer.writerows(batch_results)
                file.flush()
                os.fsync(file.fileno())

                print(f"Batch processed and saved.")
                shutil.copyfile(csv_file_path, f'/content/drive/MyDrive/Position/dog_area_percentage_1a.csv')

            except Exception as e:
                print(f"Error processing batch: {e}")

print(f"Results saved to {csv_file_path}")


Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:04<00:00, 43.1MB/s]
Processing Batches:   1%|          | 1/113 [07:27<13:55:17, 447.47s/it]

Batch processed and saved.


Processing Batches:   2%|▏         | 2/113 [07:49<6:04:20, 196.94s/it] 

Batch processed and saved.


Processing Batches:   3%|▎         | 3/113 [07:59<3:25:18, 111.99s/it]

...

Batch processed and saved.


Processing Batches:  99%|█████████▉| 112/113 [2:24:44<00:46, 46.84s/it]

Batch processed and saved.


Processing Batches: 100%|██████████| 113/113 [2:25:05<00:00, 77.04s/it]

Batch processed and saved.
Results saved to /content/drive/MyDrive/Position/dog_area_percentage_1.csv
